### Handwritten Number Recognition using Convoluted Neural Network (CNN)

Using PyTorch for Apple Silicon MPS support (Tested on M3 MacBook Air)

In [1]:
from tensorflow.keras.datasets import mnist
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Check if MPS is available
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f'Using device: {device}')

# Load MNIST dataset using TensorFlow
(x_train, y_train), (x_test, y_test) = mnist.load_data()

Using device: mps


If "MPS" appears in the output, then it utilizes Apple Silicon GPU.

In [2]:
# Normalize the data
x_train, x_test = x_train / 255.0, x_test / 255.0

# Convert to PyTorch tensors
x_train = torch.tensor(x_train, dtype=torch.float32).unsqueeze(1)  # Add channel dimension
y_train = torch.tensor(y_train, dtype=torch.long)
x_test = torch.tensor(x_test, dtype=torch.float32).unsqueeze(1)  # Add channel dimension
y_test = torch.tensor(y_test, dtype=torch.long)

# Create DataLoader
train_dataset = TensorDataset(x_train, y_train)
test_dataset = TensorDataset(x_test, y_test)

trainloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
testloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [3]:
# Define the CNN Model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = nn.ReLU()(x)
        x = self.conv2(x)
        x = nn.ReLU()(x)
        x = nn.MaxPool2d(2)(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = nn.ReLU()(x)
        x = self.fc2(x)
        return x

model = CNN().to(device)

# Define Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [4]:
# Train the Model using batches
for epoch in range(5):  # 5 epochs
    running_loss = 0.0
    correct = 0
    total = 0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
        # Calculate accuracy
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        if i % 100 == 99:    # print every 100 mini-batches
            print(f'[Epoch {epoch + 1}, Batch {i + 1}] loss: {running_loss / 100:.3f}, accuracy: {100 * correct / total:.2f}%')
            running_loss = 0.0

print('Finished Training')

[Epoch 1, Batch 100] loss: 0.609, accuracy: 81.38%
[Epoch 1, Batch 200] loss: 0.182, accuracy: 87.93%
[Epoch 1, Batch 300] loss: 0.108, accuracy: 90.90%
[Epoch 1, Batch 400] loss: 0.103, accuracy: 92.39%
[Epoch 1, Batch 500] loss: 0.078, accuracy: 93.42%
[Epoch 1, Batch 600] loss: 0.082, accuracy: 94.12%
[Epoch 1, Batch 700] loss: 0.072, accuracy: 94.64%
[Epoch 1, Batch 800] loss: 0.070, accuracy: 95.05%
[Epoch 1, Batch 900] loss: 0.065, accuracy: 95.38%
[Epoch 2, Batch 100] loss: 0.038, accuracy: 98.89%
[Epoch 2, Batch 200] loss: 0.050, accuracy: 98.75%
[Epoch 2, Batch 300] loss: 0.046, accuracy: 98.66%
[Epoch 2, Batch 400] loss: 0.030, accuracy: 98.79%
[Epoch 2, Batch 500] loss: 0.051, accuracy: 98.69%
[Epoch 2, Batch 600] loss: 0.044, accuracy: 98.70%
[Epoch 2, Batch 700] loss: 0.047, accuracy: 98.69%
[Epoch 2, Batch 800] loss: 0.039, accuracy: 98.71%
[Epoch 2, Batch 900] loss: 0.030, accuracy: 98.73%
[Epoch 3, Batch 100] loss: 0.027, accuracy: 99.05%
[Epoch 3, Batch 200] loss: 0.01

5 Epochs, 42 seconds on M3 MBA (GPU).

In [5]:
# Evaluate the Model
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct / total:.2f}%')

Accuracy of the network on the 10000 test images: 98.77%
